```{contents}
```
## RAG Evaluation 

**RAG (Retrieval-Augmented Generation) evaluation** measures whether a RAG system:

1. **Retrieves the right documents**
2. **Uses retrieved context correctly**
3. **Avoids hallucinations**
4. **Produces accurate, relevant answers**

RAG evaluation is **not just LLM evaluation**.
It evaluates **retrieval + generation together and separately**.

Supported natively in LangChain and visualized via LangSmith.

---

### Why RAG Evaluation Is Different from LLM Evaluation

| Aspect              | LLM Eval       | RAG Eval         |
| ------------------- | -------------- | ---------------- |
| Focus               | Answer quality | Answer + sources |
| Hallucination check | Weak           | Strong           |
| Retrieval quality   | ❌              | ✅                |
| Context grounding   | ❌              | ✅                |
| Explainability      | Low            | High             |

A RAG answer can be **well-written but wrong** if retrieval fails.

---

### RAG Evaluation Dimensions

RAG evaluation is usually split into **four layers**:

| Layer             | What It Measures                  |
| ----------------- | --------------------------------- |
| Retrieval Quality | Are the right docs fetched?       |
| Context Quality   | Is context sufficient & relevant? |
| Faithfulness      | Is answer grounded in context?    |
| Answer Quality    | Is the answer correct & helpful?  |

---

### RAG Evaluation Architecture

![Image](https://miro.medium.com/v2/resize%3Afit%3A1200/1%2Ak3qP8mLd2NBB5Z9S9k13-A.png)

![Image](https://cdn.prod.website-files.com/660ef16a9e0687d9cc27474a/687e38823874b267af004f9c_18_RAG_metrics-min.png)

![Image](https://d2908q01vomqb2.cloudfront.net/f1f836cb4ea6efb2a0b1b99f41ad8b103eff4b59/2024/06/06/ML-16629-rag-architecture-1.jpg)

```
Question
  ↓
Retriever ──► Retrieved Docs
  ↓
Prompt + Context
  ↓
LLM Answer
  ↓
Evaluators (Retrieval, Faithfulness, QA)
```

---

### Define a Simple RAG Pipeline

#### RAG Chain Under Test

```python
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI()

prompt = ChatPromptTemplate.from_template(
    """
    Answer the question using ONLY the context below.

    Context:
    {context}

    Question:
    {question}
    """
)

rag_chain = (
    {
        "question": lambda x: x,
        "context": retriever
    }
    | prompt
    | llm
)
```

---

### Prepare an Evaluation Dataset

#### RAG Evaluation Dataset

```python
rag_eval_data = [
    {
        "question": "What is RAG?",
        "expected_answer": "RAG combines document retrieval with text generation.",
        "expected_sources": ["retrieval augmented generation"]
    }
]
```

---

### Run RAG and Capture Outputs

#### Generate RAG Outputs

```python
results = []

for row in rag_eval_data:
    response = rag_chain.invoke(row["question"])
    docs = retriever.invoke(row["question"])

    results.append({
        "question": row["question"],
        "answer": response.content,
        "documents": docs,
        "expected": row["expected_answer"]
    })
```

---

### Retrieval Quality Evaluation

#### Check If Correct Docs Were Retrieved

```python
def retrieval_recall(docs, expected_keywords):
    text = " ".join(d.page_content.lower() for d in docs)
    return any(k in text for k in expected_keywords)

for r in results:
    score = retrieval_recall(
        r["documents"],
        ["retrieval", "generation"]
    )
    print("Retrieval recall:", score)
```

✔ Ensures **retrieval did not fail**
❌ If this fails, generation quality is irrelevant

---

### Faithfulness (Groundedness) Evaluation

#### Faithfulness Evaluator

```python
from langchain.evaluation import load_evaluator

faithfulness_eval = load_evaluator("faithfulness")

for r in results:
    score = faithfulness_eval.evaluate_strings(
        input=r["question"],
        prediction=r["answer"],
        reference="\n".join(d.page_content for d in r["documents"])
    )
    print("Faithfulness score:", score["score"])
```

✔ Detects hallucinations
✔ Confirms answer is **supported by retrieved docs**

---

### Answer Correctness Evaluation

#### Correctness Scoring

```python
correctness_eval = load_evaluator(
    "labeled_criteria",
    criteria="correctness"
)

for r in results:
    score = correctness_eval.evaluate_strings(
        input=r["question"],
        prediction=r["answer"],
        reference=r["expected"]
    )
    print("Correctness:", score["score"])
```

---

### Combined RAG Evaluation Score

#### Aggregate Metrics

```python
final_score = (
    0.4 * retrieval_score +
    0.4 * faithfulness_score +
    0.2 * correctness_score
)
```

This reflects **industry practice**:

* Retrieval & faithfulness matter more than fluency

---

### RAG Evaluation via Tracing (Production)

Enable tracing:

```bash
export LANGCHAIN_TRACING_V2=true
```

Each RAG run captures:

* Retrieved chunks
* Prompt size
* Faithfulness failures
* Latency per step
* Token cost

Reviewed in LangSmith UI.

---

### Common RAG Failure Patterns (What Evaluation Finds)

| Failure                   | Detected By     |
| ------------------------- | --------------- |
| Wrong documents retrieved | Retrieval eval  |
| Too much context          | Latency + cost  |
| Hallucinated facts        | Faithfulness    |
| Partial answers           | Correctness     |
| Outdated docs             | Retrieval audit |

---

### Best-Practice RAG Evaluation Strategy

| Stage      | What to Evaluate   |
| ---------- | ------------------ |
| Chunking   | Recall / coverage  |
| Retriever  | Precision & recall |
| Prompt     | Faithfulness       |
| Model      | Answer quality     |
| Production | Drift + regression |

---

### Mental Model

RAG evaluation answers **three questions**:

```
Did we fetch the right info?
Did the model use it?
Did it answer correctly?
```

If any answer is **no**, RAG failed.

---

### Key Takeaways

* RAG evaluation ≠ LLM evaluation
* Faithfulness is the most critical metric
* Retrieval failure guarantees generation failure
* Automated + dataset-based evaluation is mandatory
* Required for production-grade RAG systems

